In [ ]:
import os
import sys 
import json
import glob
import random
import collections
import time
import re

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import PIL.Image
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

In [ ]:
data_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'

# Set up your config

In [ ]:
DEBUG = False
mri_types = ['FLAIR','T1w','T1wCE','T2w']
SIZE = 256
NUM_IMAGES = 64

# Let's read in one dicom file to show you what I'm talking about.

# Scenario 1: Read it WITHOUT applying voi_lut

In [ ]:
files = sorted(glob.glob(f"{data_directory}/train/00688/FLAIR/*.dcm"), 
               key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

In [ ]:
slices = []
for filepath in tqdm(files):
    dicom = pydicom.read_file(filepath)
    data = dicom.pixel_array
    slices.append(data)
    
# Also, get the plane of the DICOM - axial, sagittal, or coronal.
x1, y1, _, x2, y2, _ = [round(j) for j in dicom.ImageOrientationPatient]
cords = [x1, y1, x2, y2]

if cords == [1, 0, 0, 0]:
    plane = 'Coronal'
elif cords == [1, 0, 0, 1]:
    plane = 'Axial'
elif cords == [0, 1, 0, 0]:
    plane = 'Sagittal'
else:
    plane = 'Unknown'
print(plane)

In [ ]:
img3d = np.stack(slices)

In [ ]:
# Normalize it so we can plot it
if img3d.sum() != 0:
    img3d = img3d - np.min(img3d)
    img3d = img3d / np.max(img3d)
    img3d = (img3d * 255).astype(np.uint8)

In [ ]:
print(img3d.shape)

In [ ]:
# The coronal view of the scan
plt.imshow(img3d[len(img3d)//2, :, :])

In [ ]:
# The axial view of the scan
plt.imshow(img3d[:, img3d.shape[1]//2, :])

### Quickly plot max values for each slice on the axial view

In [ ]:
maxs = []
q25s = []
for slice_idx in tqdm(range(len(img3d))):
    maxs.append(np.max(img3d[slice_idx, img3d.shape[1]//2, :]))
    q25s.append(np.quantile(img3d[slice_idx, img3d.shape[1]//2, :], 0.25))
    
ax = sns.lineplot(x = range(len(img3d)), y = maxs, color='orange', label='Max values')
sns.lineplot(x = range(len(img3d)), y=q25s, color='blue', label='Q25 values')

# Scenario 2: Repeat the same code, but applying VOI LUT.

In [ ]:
slices = []
for filepath in tqdm(files):
    dicom = pydicom.read_file(filepath)
    data = apply_voi_lut(dicom.pixel_array, dicom)
    slices.append(data)
    
# Also, get the plane of the DICOM - axial, sagittal, or coronal.
x1, y1, _, x2, y2, _ = [round(j) for j in dicom.ImageOrientationPatient]
cords = [x1, y1, x2, y2]

if cords == [1, 0, 0, 0]:
    plane = 'Coronal'
elif cords == [1, 0, 0, 1]:
    plane = 'Axial'
elif cords == [0, 1, 0, 0]:
    plane = 'Sagittal'
else:
    plane = 'Unknown'
print(plane)

In [ ]:
img3d = np.stack(slices)
# Normalize it so we can plot it
if img3d.sum() != 0:
    img3d = img3d - np.min(img3d)
    img3d = img3d / np.max(img3d)
    img3d = (img3d * 255).astype(np.uint8)

In [ ]:
print(img3d.shape)

In [ ]:
# The coronal view of the scan
plt.imshow(img3d[len(img3d)//2, :, :])

In [ ]:
# The axial view of the scan
plt.imshow(img3d[:, img3d.shape[1]//2, :])

### Quickly plot max values for each slice

In [ ]:
maxs = []
q25s = []
for slice_idx in tqdm(range(len(img3d))):
    maxs.append(np.max(img3d[slice_idx, img3d.shape[1]//2, :]))
    q25s.append(np.quantile(img3d[slice_idx, img3d.shape[1]//2, :], 0.25))
    
ax = sns.lineplot(x = range(len(img3d)), y = maxs, color='orange', label='Max values')
sns.lineplot(x = range(len(img3d)), y=q25s, color='blue', label='Q25 values')

# What is going on with those bands?? only appears with voi_lut

# Is it a matplotlib issue? Try changing the figsize

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
ax.imshow(img3d[:, img3d.shape[1]//2, :])

In [ ]:
fig, ax = plt.subplots(figsize=(24, 24))
ax.imshow(img3d[:, img3d.shape[1]//2, :])

# Does it exist with other colormaps?

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(img3d[:, img3d.shape[1]//2, :], cmap='gray')